# We'll learn how to calculate summary statistics on subsets of a database table. 

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql


In [2]:
conn = sql.connect("jobs.db")

In [3]:
# Creating a function to read query
def read_query(query):
    return pd.read_sql_query(query, conn)

In [4]:
# query that returns the number of majors with mostly male students.

q = '''SELECT COUNT(Major) FROM recent_grads
    WHERE ShareWomen <0.5''';

read_query(q)

,COUNT(Major)
0,76


SQLite returned a table with a column `(COUNT(Major))` and the count as a row in that column (`76`).

A key idea in `SQL` is that everything is a **table**.

Functions like `COUNT()` are known as **aggregate** functions. Aggregate functions are applied over columns of values and return a single value. `MIN()` and `MAX()`, for example, calculate and return the minimum and maximum values in a column.

One thing to note is that while `COUNT()` can be used on any column (because it's just counting the number of values), the other aggregate functions (`MIN()`, `MAX()`, etc) can only be used on numeric columns (since these arithmetic calculations only work with numbers).

In [5]:
# query that returns the lowest median salary from the major category Engineering

q = '''SELECT Major, Major_category, min(Median) 
    FROM recent_grads 
    WHERE Major_category = "Engineering" ''';
read_query(q)

,Major,Major_category,min(Median)
0,ARCHITECTURE,Engineering,40000


The final two aggregation functions we'll look at are `SUM()` and `AVG()`. Applying the `SUM()` function will add all of the values in a column while `AVG()` will compute the average. Lastly, the `TOTAL()` function also returns the sum as a floating point value (even if the column contains integers). The `TOTAL()` function should be used when working with a column containing floating point values.

In [6]:
q = "Select SUM(Total) from recent_grads"
read_query(q)

,SUM(Total)
0,6776015


We can actually write queries that answer multiple questions at once.

In [7]:
q = 'SELECT AVG(Total), MIN(Men), MAX(Women) FROM recent_grads'

read_query(q)

,AVG(Total),MIN(Men),MAX(Women)
0,39167.716763,119,307087


In [8]:
q = 'SELECT round(AVG(Total),2), MIN(Men), MAX(Women) FROM recent_grads'

read_query(q)

,"round(AVG(Total),2)",MIN(Men),MAX(Women)
0,39167.72,119,307087


All of the queries we've written so far have had somewhat unpleasant column names in the results, like `AVG(SUM)` and `MIN(Men)`. 

Many companies use SQL environments and tools that can run our query, turn the results into a plot of our choosing, and then create a PDF report containing multiple plots

In [9]:
# query that returns
# 1.the number of rows as Number of Students
# 2.the maximum value of Unemployment_rate as Highest Unemployment Rate

q = '''SELECT count(*) "Number of Students", 
MAX(Unemployment_rate) as "Highest Unemployment Rate" FROM recent_grads'''
read_query(q)

,Number of Students,Highest Unemployment Rate
0,173,0.177226


We can reference renamed columns when writing longer queries to make our code more compact

In [10]:
q = """SELECT Major m, Major_category mc, Unemployment_rate ur
FROM recent_grads
WHERE (mc = 'Engineering') AND (ur > 0.04 and ur < 0.08)
ORDER BY ur DESC"""
read_query(q)

,m,mc,ur
0,GEOLOGICAL AND GEOPHYSICAL ENGINEERING,Engineering,0.075038
1,MISCELLANEOUS ENGINEERING,Engineering,0.074393
2,CIVIL ENGINEERING,Engineering,0.070610
3,COMPUTER ENGINEERING,Engineering,0.065409
4,AEROSPACE ENGINEERING,Engineering,0.065162
5,ARCHITECTURAL ENGINEERING,Engineering,0.061931
6,CHEMICAL ENGINEERING,Engineering,0.061098
7,GENERAL ENGINEERING,Engineering,0.059824
8,ELECTRICAL ENGINEERING,Engineering,0.059174
9,MECHANICAL ENGINEERING,Engineering,0.057342


We can return all of the unique values in a column using the `DISTINCT` statement.

We can use the `DISTINCT` statement with multiple columns to return unique pairings of those columns

In [11]:
q = 'SELECT DISTINCT Major_category FROM recent_grads'
read_query(q)

,Major_category
0,Engineering
1,Business
2,Physical Sciences
3,Law & Public Policy
4,Computers & Mathematics
5,Agriculture & Natural Resources
6,Industrial Arts & Consumer Services
7,Arts
8,Health
9,Social Science


In [12]:
q = '''SELECT DISTINCT Major, Major_category FROM recent_grads limit 5'''
read_query(q)

,Major,Major_category
0,PETROLEUM ENGINEERING,Engineering
1,MINING AND MINERAL ENGINEERING,Engineering
2,METALLURGICAL ENGINEERING,Engineering
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering
4,CHEMICAL ENGINEERING,Engineering


In [13]:
q = """SELECT COUNT(DISTINCT(Major_category)) unique_major_categories FROM recent_grads"""
read_query(q)

,unique_major_categories
0,16


In [14]:
q = '''SELECT COUNT(DISTINCT(Major)) as unique_majors, COUNT(DISTINCT(Major_category)) unique_major_categories,
    COUNT(DISTINCT(Major_code)) unique_major_codes FROM recent_grads'''

read_query(q)

,unique_majors,unique_major_categories,unique_major_codes
0,173,16,173


SQL supports the standard arithmetic operators: `*`, `+`, `-`, and `/`, and we can use them like any other operator:

One thing to note is that multiplying or dividing columns with a floating point value (or a column with floating point values) will result in floating point values:

In [15]:
# query that computes the difference between the 25th and 75th percentile of salaries for all majors

q = '''SELECT Major, Major_category, (P75th - P25th) quartile_spread 
       From recent_grads
       Order by quartile_spread ASC LIMIT 20'''

read_query(q)

,Major,Major_category,quartile_spread
0,MILITARY TECHNOLOGIES,Industrial Arts & Consumer Services,0
1,SCHOOL STUDENT COUNSELING,Education,2000
2,LIBRARY SCIENCE,Education,2000
3,COURT REPORTING,Law & Public Policy,4000
4,PHARMACOLOGY,Biology & Life Science,5000
5,EDUCATIONAL ADMINISTRATION AND SUPERVISION,Education,6000
6,COUNSELING PSYCHOLOGY,Psychology & Social Work,6800
7,SPECIAL NEEDS EDUCATION,Education,10000
8,MATHEMATICS TEACHER EDUCATION,Education,10000
9,SOCIAL WORK,Psychology & Social Work,10000


The `LENGTH` function returns the number of characters in the input strings. We'll use the `LOWER` function to replace the `Major` column with one where all values are written in lowercase.

In [16]:
q = """SELECT 'Major: ' || LOWER(Major) AS Major,
       Total, Men, Women, Unemployment_rate,
       LENGTH(Major) AS Length_of_name
  FROM recent_grads
 ORDER BY Unemployment_rate DESC;"""

read_query(q)

,Major,Total,Men,Women,Unemployment_rate,Length_of_name
0,Major: nuclear engineering,2573,2200,373,0.177226,19
1,Major: public administration,5629,2947,2682,0.159491,21
2,Major: computer networking and telecommunications,7613,5291,2322,0.151850,42
3,Major: clinical psychology,2838,568,2270,0.149048,19
4,Major: public policy,5978,2695,905,0.128426,13
...,...,...,...,...,...,...
168,Major: mathematics and computer science,609,803,10320,0.000000,32
169,Major: botany,1329,626,703,0.000000,6
170,Major: soil science,685,4266,13843,0.000000,12
171,Major: educational administration and supervision,804,78253,63698,0.000000,42
